Create a test sql database from titanic dataset.

https://python.langchain.com/docs/use_cases/sql/csv/

In [1]:
import pandas as pd
from pyprojroot import here

In [2]:
df = pd.read_csv(here("data/for_upload/titanic.csv"))
print(df.shape)
print(df.columns.tolist())
display(df.head(3))

(887, 8)
['Survived', 'Pclass', 'Name', 'Sex', 'Age', 'Siblings/Spouses Aboard', 'Parents/Children Aboard', 'Fare']


,Survived,Pclass,Name,Sex,Age,Siblings/Spouses Aboard,Parents/Children Aboard,Fare
0,0,3,Mr. Owen Harris Braund,male,22.0,1,0,7.2500
1,1,1,Mrs. John Bradley (Florence Briggs Thayer) Cum...,female,38.0,1,0,71.2833
2,1,3,Miss. Laina Heikkinen,female,26.0,0,0,7.9250


### **SQL**

Using SQL to interact with CSV data is the recommended approach because it is easier to limit permissions and sanitize queries than with arbitrary Python.

Most SQL databases make it easy to load a CSV file in as a table (DuckDB, SQLite, etc.). Once you’ve done this you can use all of the chain and agent-creating techniques outlined in the SQL use case guide. Here’s a quick example of how we might do this with SQLite:

In [8]:
from langchain_community.utilities import SQLDatabase
from sqlalchemy import create_engine
db_path = str(here("data")) + "/test_sqldb.db"
db_path = f"sqlite:///{db_path}"

engine = create_engine(db_path)
# df.to_sql("titanic", engine, index=False)
df.to_sql("titanic", engine, index=False)

887

In [6]:
db_path

'sqlite:///d:\\Data Science\\LLM\\Youtube\\chat-with-sql-csv\\Advanced-QA-and-RAG-Series\\Q&A-and-RAG-with-SQL-and-TabularData\\data/test_sqldb.db'

In [18]:
# Add cancer.csv into db
df2=pd.read_csv(here("data/csv_xlsx/cancer.csv"))
df2.to_sql("cancer", engine, index=False)

101

In [17]:
df2.head()

,Id,clump_thickness,uniformity_of_cell_size,uniformity_of_cell_shape,marginal_adhesion,single_epithelial_cell_size,bare_nuclei,bland_chromatin,normal_nucleoli,mitosis
0,1,4,1,1,1,2,1,1,1,1
1,2,1,1,2,1,2,1,2,1,1
2,3,3,1,1,1,1,1,2,1,1
3,4,6,1,1,3,2,1,1,1,1
4,5,6,1,1,1,1,1,1,1,1


For multiple csv files, we can create a sql with multiple tables:
```
df1.to_sql("csv1_name", engine, index=False)
df2.to_sql("csv2_name", engine, index=False)
```

In [19]:
db = SQLDatabase(engine=engine)
print(db.dialect)
print(db.get_usable_table_names())
# db.run("SELECT * FROM titanic WHERE Age < 2;")

sqlite
['cancer', 'titanic']


In [12]:
db.dialect

'sqlite'

In [22]:
print(db.run("SELECT * FROM titanic WHERE Age < 2;"))

[(1, 2, 'Master. Alden Gates Caldwell', 'male', 0.83, 0, 2, 29.0), (0, 3, 'Master. Eino Viljami Panula', 'male', 1.0, 4, 1, 39.6875), (1, 3, 'Miss. Eleanor Ileen Johnson', 'female', 1.0, 1, 1, 11.1333), (1, 2, 'Master. Richard F Becker', 'male', 1.0, 2, 1, 39.0), (1, 1, 'Master. Hudson Trevor Allison', 'male', 0.92, 1, 2, 151.55), (1, 3, 'Miss. Maria Nakid', 'female', 1.0, 0, 2, 15.7417), (0, 3, 'Master. Sidney Leonard Goodwin', 'male', 1.0, 5, 2, 46.9), (1, 3, 'Miss. Helene Barbara Baclini', 'female', 0.75, 2, 1, 19.2583), (1, 3, 'Miss. Eugenie Baclini', 'female', 0.75, 2, 1, 19.2583), (1, 2, 'Master. Viljo Hamalainen', 'male', 0.67, 1, 1, 14.5), (1, 3, 'Master. Bertram Vere Dean', 'male', 1.0, 1, 2, 20.575), (1, 3, 'Master. Assad Alexander Thomas', 'male', 0.42, 0, 1, 8.5167), (1, 2, 'Master. Andre Mallet', 'male', 1.0, 0, 2, 37.0042), (1, 2, 'Master. George Sibley Richards', 'male', 0.83, 1, 1, 18.75)]


**Equivalent in Pandas**

In [20]:
df[df["Age"]<2]

,Survived,Pclass,Name,Sex,Age,Siblings/Spouses Aboard,Parents/Children Aboard,Fare
77,1,2,Master. Alden Gates Caldwell,male,0.83,0,2,29.0000
163,0,3,Master. Eino Viljami Panula,male,1.00,4,1,39.6875
171,1,3,Miss. Eleanor Ileen Johnson,female,1.00,1,1,11.1333
182,1,2,Master. Richard F Becker,male,1.00,2,1,39.0000
303,1,1,Master. Hudson Trevor Allison,male,0.92,1,2,151.5500
379,1,3,Miss. Maria Nakid,female,1.00,0,2,15.7417
384,0,3,Master. Sidney Leonard Goodwin,male,1.00,5,2,46.9000
466,1,3,Miss. Helene Barbara Baclini,female,0.75,2,1,19.2583
641,1,3,Miss. Eugenie Baclini,female,0.75,2,1,19.2583
751,1,2,Master. Viljo Hamalainen,male,0.67,1,1,14.5000


### **Create an agent to interact with the Database**

In [24]:
import os
from dotenv import load_dotenv
import warnings
warnings.filterwarnings("ignore")
print("Environment variables are loaded:", load_dotenv())
print("test by reading a variable:", os.getenv("OPENAI_API_TYPE"))

Environment variables are loaded: True
test by reading a variable: None


In [8]:
# from langchain.chat_models import AzureChatOpenAI

# model_name = os.getenv("gpt_deployment_name")
# azure_openai_api_key = os.environ["OPENAI_API_KEY"]
# azure_openai_endpoint = os.environ["OPENAI_API_BASE"]
# llm = AzureChatOpenAI(
#     openai_api_version=os.getenv("OPENAI_API_VERSION"),
#     azure_deployment=model_name,
#     model_name=model_name,
#     temperature=0.0)

/home/farzad_rzt/python_env/sql-rag-gpt/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `AzureChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import AzureChatOpenAI`.
  warn_deprecated(


In [29]:
# Instead Using AzureOpenai, Try using  OpenAI
from langchain_openai import ChatOpenAI
load_dotenv()

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

llm = ChatOpenAI(model = "gpt-4o-mini",
                 temperature=0.3)

In [59]:
# Create OpenSource LLM Model
from langchain_ollama import ChatOllama
llm_ollama = ChatOllama(model="sqlcoder:15b", temperature=0.5)

In [ ]:
from langchain.llms import HuggingFacePipeline
from transformers import pipeline
#Create OpenSource LLM Model from Hugging Face
# But it Takes BIIIGGGG Memory Better run on cloud
#Set Up Enviroment
load_dotenv(override=True)
api_key = os.getenv('HF_TOKEN')

# Load SQL-compatible model (replace with the model you prefer)
model_name = "defog/sqlcoder-7b-2" # Example: SQLCoder
hf_pipeline = pipeline("text-generation", model=model_name, device=0)  # Use GPU if available

# Wrap model in LangChain's HuggingFace wrapper
llm_hf = HuggingFacePipeline(pipeline=hf_pipeline)

In [30]:
llm.invoke("Hello, hi there")

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 11, 'total_tokens': 21, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_72ed7ab54c', 'finish_reason': 'stop', 'logprobs': None}, id='run-2a2be111-8cb0-41d7-8f51-452fffdc2491-0', usage_metadata={'input_tokens': 11, 'output_tokens': 10, 'total_tokens': 21, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [67]:
from langchain_community.agent_toolkits import create_sql_agent
# agent_executor = create_sql_agent(llm, db=db, agent_type="openai-tools", verbose=False)
agent_executor = create_sql_agent(llm_ollama, agent_type='zero-shot-react-description', db=db, verbose=True)

In [68]:
agent_executor.invoke({"input": "What is the rate of survival of Woman in Titanic db"}).get("output")



> Entering new SQL Agent Executor chain...


KeyboardInterrupt: 

In [45]:
agent_executor.invoke({"input": "what's the average age of survivors"}).get("output")

'The average age of survivors is approximately 28.41 years.'

In [37]:
# Equivalence in Pandas
df[df["Survived"]==1]["Age"].mean()

28.408391812865496